# AiDM assignment 1
## Recommender system --- Matrix factorization 

### Read data
We load the ratings data into a matrix which has 4 columns. The first column gives the user id. The second column gives the movie id. The third column gives the rating which can only be a integer from 1 to 5. The fourth column gives the timestamp in a unit of second.

In [1]:
import numpy as np
data = np.genfromtxt('ml-1m/ratings.dat', delimiter= '::')

## Gradient descent
Next we create a list containing the unique user and movie ids, as well as a vector of 20 weights for each. We will train the network so that the dot product of a given user vector with a movie vector will be a prediction of the user's rating for that movie. In this way, the weights of a movie might reflect certain qualities whereas the weights of a user might reflect which qualities that particular user prefers.

In [2]:
user_list = np.unique(data[:,0])
movie_list = np.unique(data[:,1])
user_vector = np.random.uniform(size=(len(user_list),20))
movie_vector = np.random.uniform(size=(len(movie_list),20))

Next we create vectors of the indicies for the weights vectors for the users and movies in the original data from ratings.dat. This is important since weight vectors for *unique* movies is not the same as the movie id, since there some movies which have not been rated in the data and it would be unhelpful to have random, unconstrained wieght vectors for movies not rated in the data.

In [7]:
user_locator = np.zeros(len(data))
movie_locator = np.zeros(len(data))
for i in range(len(data)):
    user_locator[i] = np.where(user_list==data[i,0])[0][0]
    movie_locator[i] = np.where(movie_list==data[i,1])[0][0]
print(user_locator)
print(movie_locator)
print(data[:,1]) # Not the same! There are some unrated movies.

[   0.    0.    0. ... 6039. 6039. 6039.]
[1104.  639.  853. ...  548. 1024. 1025.]
[1193.  661.  914. ...  562. 1096. 1097.]


In [52]:
lrate = 0.001
lamb = 0.01
for count in range(10):
    for i in range(len(data)):
        est_rating = np.dot(user_vector[int(user_locator[i])],movie_vector[int(movie_locator[i])])
        if est_rating < 1 :
            est_rating = 1
        if est_rating > 5 :
            est_rating = 5
        error = data[i,2] - est_rating
        user_vector[int(user_locator[i])]  += \
        lrate * (error * movie_vector[int(movie_locator[i])] - lamb *  user_vector[int(user_locator[i])])
        movie_vector[int(movie_locator[i])] += \
        lrate * (error * user_vector[int(user_locator[i])] - lamb * movie_vector[int(movie_locator[i])])
    count += 1

In [58]:
np.dot(user_vector[int(user_locator[1])],movie_vector[int(movie_locator[1])])

3.3102970966053937

In [ ]:
print("Test Git Push")